In [ ]:
!apt-get -qq update
!apt-get -qq install -y git-lfs libgl1-mesa-glx libglfw3 > /dev/null
!git lfs install
!pip install mujoco matplotlib imageio

import os
import shutil

if os.path.exists("/content/air_hockey_challenge"):
    shutil.rmtree("/content/air_hockey_challenge")

# リポジトリをclone
!git clone https://github.com/AirHockeyChallenge/air_hockey_challenge.git
%cd air_hockey_challenge
!git checkout tournament
!git lfs pull

# ✅✅✅
print("ここで一旦ストップ！！")
print("左のファイルタブを開いて、")
print("/content/ に table_rim.stl をアップロード")


In [ ]:
!pip install -q mujoco mediapy
!apt-get -qq update
!apt-get -qq install -y ffmpeg


import os
import subprocess

# GPU確認
if subprocess.run('nvidia-smi', shell=True).returncode:
  raise RuntimeError('GPUがColabで有効になっていません。ランタイムのタイプをGPUに変更してください。')

# EGL設定
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

print('Setting environment variable MUJOCO_GL=egl')
%env MUJOCO_GL=egl

import mujoco
import mediapy as media
import numpy as np
import matplotlib.pyplot as plt

print("✅ セットアップ完了！")


In [3]:
#ランタイムGPUにしないと動かない
!mv /content/table_rim.stl /content/air_hockey_challenge/air_hockey_challenge/environments/data/iiwas/assets/

import os
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import imageio
import tempfile

# 作業ディレクトリを移動
os.chdir("/content/air_hockey_challenge/air_hockey_challenge/environments/data")

# table.xml をパッチ（fileパス修正）
with open("table.xml", "r") as f:
    xml_text = f.read()

xml_text = xml_text.replace('file="table_rim.stl"', 'file="iiwas/assets/table_rim.stl"')

with open("patched_table.xml", "w") as f:
    f.write(xml_text)

# モデル読み込み
model = mujoco.MjModel.from_xml_path("patched_table.xml")
data = mujoco.MjData(model)


duration = 2.0  # シミュレーション時間 (seconds)
framerate = 30  # フレームレート (fps)
height = 480
width = 640

frames = []
mujoco.mj_resetData(model, data)  # 状態と時間をリセット

with mujoco.Renderer(model, height=height, width=width) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # 1ステップ進める

        # 必要なだけフレームを保存
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)
            pixels = renderer.render()
            frames.append(pixels)

import IPython.display as display
import PIL.Image

# 一時ファイルにmp4動画を書き出す
with tempfile.NamedTemporaryFile(suffix='.mp4') as f:
    imageio.mimwrite(f.name, frames, fps=framerate, macro_block_size=None)
    display.display(display.Video(f.name, embed=True))
